## Submissão básica

- Neste notebook vamos criar uma submissão <strong>baseline</strong>, que é uma submissão inicial sem muita análise e processamento.
- A ideia é entender como treinar um modelo simples com os dados e criar uma submissão válida, que será nosso ponto de partida a ser superado.
- Também serve para verificar se nossa validação local está semelhante à pontuação na competição. Como a quantidade de submissões ao longo da competição é limitada em 300, o ideal é passarmos a maior parte do tempo validando os experimentos localmente.
- Ou seja, é esencial que a acurácia obtida na validação local se aproxime da acurácia do leaderboard.

In [1]:
import re
import numpy as np
import pandas as pd

from sklearn.model_selection import cross_val_score
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.svm import *
from sklearn.linear_model import *

In [2]:
# Treino, teste e modelo de submissão
train = pd.read_csv('data/Train.csv')
test = pd.read_csv('data/Test.csv')
sub = pd.read_csv('data/SampleSubmission.csv')

# Parece que os IDs do modelo de submissão estão fora de ordem
# então substituímos eles pela ordem do arquivo de teste
sub.ID = test.ID

# Target a ser previsto o treino
target = train['label']

# Dimensões dos dataframes de treino e teste
train.shape, test.shape

((70000, 3), (30000, 2))

In [3]:
# Distribuição dos targets
train.label.value_counts()

 1    38239
-1    29295
 0     2466
Name: label, dtype: int64

In [4]:
# Amostra
train.text

0        3sbaaaaaaaaaaaaaaaaaaaa lek ou le seim riahi o...
1             cha3eb fey9elkoum menghir ta7ayoul ou kressi
2        bereau degage nathef ya slim walahi ya7chiw fi...
3                                                ak slouma
4        entom titmanou lina a7na 3iid moubarik a7na ch...
                               ...                        
69995    pff bayna beli kbira f wejhakk yakhiii rouhi r...
69996    aman lmara jeya zidou t3am9ou fel a7deeth akth...
69997                                winha nakhtabha hhhhh
69998                                  fachel enta w houwa
69999       nchla lyouma nesm3ou a5beer bahiya 3la jem3iya
Name: text, Length: 70000, dtype: object

In [5]:
train.head()

,ID,text,label
0,13P0QT0,3sbaaaaaaaaaaaaaaaaaaaa lek ou le seim riahi o...,-1
1,SKCLXCJ,cha3eb fey9elkoum menghir ta7ayoul ou kressi,-1
2,V1TVXIJ,bereau degage nathef ya slim walahi ya7chiw fi...,-1
3,U0TTYY8,ak slouma,1
4,68DX797,entom titmanou lina a7na 3iid moubarik a7na ch...,-1


In [6]:
# Preprocessamento básico

def preprocess(txt):   
    txt = re.sub(r' +', ' ', txt)
    txt = txt.lower()
    return txt

train['txt_ok'] = train.text.apply(preprocess)
test['txt_ok'] = test.text.apply(preprocess)

In [7]:
# Todo o conjunto de textos
corpus = pd.concat([
    train['txt_ok'],
    test['txt_ok']
], axis=0)

In [8]:
# Vetorização: transformação dos textos em uma matriz numérica
# de documentos x termos.
vec = TfidfVectorizer()
vec.fit(corpus)
train_vec = vec.transform(train['txt_ok'])
test_vec = vec.transform(test['txt_ok'])

In [9]:
# Repare que a matriz gerada é esparsa, o que permite lidar
# com as 189093 colunas sem ocupar muita memória
train_vec

<70000x189093 sparse matrix of type '<class 'numpy.float64'>'
	with 613395 stored elements in Compressed Sparse Row format>

In [15]:
pd.Series(vec.get_feature_names())#.str.len()

0             00
1            000
2           0000
3          00000
4         000000
           ...  
189088    ȝzzǿùz
189089      ȝàßß
189090      əlli
189091      ١vue
189092    ٣rouby
Length: 189093, dtype: object

In [24]:
# Validação cruzada estratificada 5-folds com um modelo linear
m = LinearSVC(random_state=42)

s = cross_val_score(
    m,
    train_vec, 
    target, 
    scoring='accuracy',
    cv=5,
    n_jobs=-1
)

# Média das 5 validações
print(np.round(s, 5))
print('Mean: ', np.mean(s))

[0.79079 0.78929 0.78993 0.789   0.79257]
Mean:  0.7903142857142857


In [12]:
train_vec

<70000x189093 sparse matrix of type '<class 'numpy.float64'>'
	with 613395 stored elements in Compressed Sparse Row format>

In [23]:
# Treina com todos os dados e gera uma submissão
m.fit(train_vec, target)

sub.label = m.predict(test_vec)

sub.to_csv('sub.csv', index=False)
sub

,ID,label
0,2DDHQW9,1
1,5HY6UEY,-1
2,ATNVUJX,1
3,Q9XYVOQ,-1
4,TOAHLRH,1
...,...,...
29995,NHXTL3R,-1
29996,U1YWB2O,1
29997,O3KYLM0,1
29998,W4C38TY,-1



---

In [153]:
import torch
import numpy as np

In [145]:
output = torch.FloatTensor([0,0,0,1])
target = torch.LongTensor([3])

loss = torch.nn.functional.cross_entropy(output.view(1, -1), target)
print(loss)

tensor(0.7437)


In [129]:
# Softmax
output.exp() / output.exp().sum(), torch.nn.functional.softmax(output, dim=0)

(tensor([0.1749, 0.1749, 0.1749, 0.4754]),
 tensor([0.1749, 0.1749, 0.1749, 0.4754]))

In [134]:
-torch.log(torch.tensor(0.4754))

tensor(0.7436)

In [137]:
output.exp().sum().log()

tensor(-0.7437)

In [146]:
#nll_loss(log_softmax(input, 1), target

In [182]:
p = [0.10, 0.40, 0.50]
q = [0.80, 0.15, 0.05]

p, q = np.array(p), np.array(q)

In [183]:
def cross_entropy(p, q):
    return np.round(-(p * np.log2(q)).sum(), 3)

In [184]:
cross_entropy(p, p), cross_entropy(q, q), cross_entropy(p, q)

(1.361, 0.884, 3.288)

In [185]:
def kl_divergence(p, q):
    return (p * np.log2(p / q)).sum()
 
# calculate entropy H(P)
def entropy(p):
    return -(p * np.log2(p)).sum()
 
# calculate cross entropy H(P, Q)
def cross_entropy2(p, q):
    return np.round(entropy(p) + kl_divergence(p, q), 3)
 

In [186]:
cross_entropy2(p, p), cross_entropy2(q, q), cross_entropy2(p, q)

(1.361, 0.884, 3.288)

In [187]:
entropy(p), entropy(q)

(1.360964047443681, 0.8841837197791889)

In [196]:
x = 5

np.log(1 + np.exp(2))

2.1269280110429727